# Vector Only RAG

In this notebook we are going to explore how to make a vector only RAG system using the `DocumentMemory`. RAG stands for Retrieval Augmented Generation, the basic idea of this technique is to not use the LLM to recall knowledge but instead rely on a external system like a database to provide information to the system.

The main advantage of vector-only RAG systems over Graph RAG is the fact that you don't need to model your knowledge and can use very simple data pipelines. They excel in retrieving information based on the context of the question but fail short in retrieving factual knowledge (you've been warned).

In this tutorial we are going to present you a small system that is nice to learn how RAG works in practice. To make things bit more complicated, we are going to start from the PDF.

In [1]:
import pymupdf # Used to extract the text from our PDF
from hybridagi.core.datatypes import Document, DocumentList
from hybridagi.core.pipeline import Pipeline
from hybridagi.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from hybridagi.modules.splitters.document_sentence_splitter import DocumentSentenceSplitter
from hybridagi.modules.embedders.document_embedder import DocumentEmbedder

filename = "data/SpelkeKinzlerCoreKnowledge.pdf"
pdf_document = pymupdf.open(filename)

input_docs = DocumentList()

embeddings = SentenceTransformerEmbeddings(
    model_name_or_path = "all-MiniLM-L6-v2",
    dim = 386, # The dimention of the embeddings vector (also called dense vector)
)
# We are going to create a full document that have all the pages
# With the increase of LLM context windows nowadays, this can be helpfull
full_document = Document(text="", metadata={"filename": filename})
for page_nb, page in enumerate(pdf_document): # iterate over the document pages
    text = page.get_text()
    full_document.text += text
    input_docs.docs.append(Document(
        text=text,
        metadata={"filename": filename, "page": page_nb},
        parent_id=full_document.id))

# Now that we have our input documents, we can start to make our data processing pipeline

pipeline = Pipeline()

pipeline.add("chunk_documents", DocumentSentenceSplitter())
pipeline.add("embed_chunks", DocumentEmbedder(embeddings=embeddings))

output_docs = pipeline(input_docs)

/home/yoan/.cache/pypoetry/virtualenvs/hybridagi-B1GoJrSC-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Saving the documents into memory

Now that we have our documents we can load them into memory, for storing unstructured textual documents, we provide the `DocumentMemory` and for this example we are going to use the local integration for rapid prototyping.

In [2]:
from hybridagi.memory.integration.local import LocalDocumentMemory

document_memory = LocalDocumentMemory(index_name="simple_rag")

document_memory.update(full_document)
document_memory.update(input_docs)
document_memory.update(output_docs)

document_memory.show() # Let's see what the memory look like now

simple_rag_document_memory.html


#### Retrieving the document chunks